## Question-2

In [ ]:
# @title include the libraries
import pandas as pd
import numpy as np
from numpy import genfromtxt
import pdb
from matplotlib import pyplot as plt

In [ ]:
# @title Load the training Data
import numpy as np

def ReadX(path):
    print(f'>>> Reading data from: {path} ...')
    with open(path) as f:
        # only one line that includes everything
        file = f.readlines()

    print(f'#instances: {len(file)}') # 7352 for training set, 2947 for test set

    X_all = []
    for instance in file:
        f = filter(None, instance.split(' '))
        instance_filterd = list(f)
        instance_cleaned = [float(attr.strip()) for attr in instance_filterd]
        X_all.append(instance_cleaned)
    X_all = np.array(X_all)
    print('>>> Reading finished! Data are converted to numpy array.')
    print(f'shape of X: {X_all.shape} ==> each instance has {X_all.shape[1]} attributes.')

    return X_all

def ReadY(path):
    print(f'>>> Reading data from: {path} ...')
    with open(path) as f:
        # only one line that includes everything
        file = f.readlines()

        print(f'#instances: {len(file)}')  # 7352 for training set, 2947 for test set

    y_all = [float(label.strip()) for label in file]
    y_all = np.array(y_all)
    print('>>> Reading finished! Data are converted to numpy array.')
    print(f'shape of y: {y_all.shape}')
    return y_all



# You can change the path of the files
X_train = ReadX('/content/activity_X_train.txt')
y_train = ReadY('/content/activity_y_train.txt')

X_test = ReadX('/content/activity_X_test.txt')
y_test = ReadY('/content/activity_y_test.txt')


# num_features = X_train.shape[0]
# num_training_example = training_data.shape[1]
# num_training_labels = training_labels.shape[0]
# print("num of features =",num_features,"num_training_example=",num_training_example)
# print("num_labels=",num_training_labels)

In [ ]:
# @title  Implementation of perceptron Algorithm (Binary Classification)
def perceptron_algo(X:np.asarray,y:np.asarray,max_pass:int=500, w:np.asarray=np.zeros((57,1)),b:np.float32=0):
  mistakes = np.zeros((0)) # to record the mistakes in each iteration
  num_features = X.shape[0]
  num_training_example = X.shape[1]
  for j in range(max_pass):
    mistake_t = 0
    for i in range(num_training_example):
      if(y[i]*(X[:,i][...,np.newaxis].T@w+b).squeeze() <= 0) :
        w = w + y[i]*X[:,i][...,np.newaxis]
        b = b + y[i]
        mistake_t+=1
    mistakes = np.hstack((mistakes,np.asarray([mistake_t])))
    #print(f"mistake-at-{j}",mistake_t)
  return w,b,mistakes



In [ ]:
# @title Multiclass-Perceptron (One Vs All), This calls the perceptron_algo for each label
def perceptron_algo_multiclass(X:np.asarray,y:np.asarray,w,b,max_pass:int=500,num_classes:int=1):
  mistakes = np.zeros((num_classes,max_pass))
  w_final = np.zeros((num_classes,X.shape[0],1))
  b_final = np.zeros((num_classes,))
  for i in range(num_classes):
    print("Training the CLASS:",i+1)
    y_i = np.ones([X.shape[1],])
    idx = np.argwhere(y != i+1)
    y_i[idx] = -1
    w_i,b_i,mistake_i = perceptron_algo(X,y_i,max_pass=500, w=w[i,:,:],b=b[i])
    w_final[i,:,:] = w_i
    b_final[i] = b_i
    mistakes[i] = mistake_i
    print(f"final-mistake for class-{i+1}:=",mistake_i[-1])
  return w_final,b_final,mistakes

In [ ]:
# @title Calling Perceptron Algorithm
X = X_train.T
y = y_train
max_pass_itr = 500
num_classes = 6
num_features = X.shape[0]
w,b,mistakes = perceptron_algo_multiclass(X,y,w=np.zeros((num_classes,num_features,1)),b=np.zeros((num_classes,)) , max_pass=max_pass_itr,num_classes=6)

Training the CLASS: 1
final-mistake for class-1:= 0.0
Training the CLASS: 2
final-mistake for class-2:= 0.0
Training the CLASS: 3
final-mistake for class-3:= 0.0
Training the CLASS: 4
final-mistake for class-4:= 40.0
Training the CLASS: 5
final-mistake for class-5:= 38.0
Training the CLASS: 6
final-mistake for class-6:= 0.0


In [ ]:
# @title Calculating Metrices
total_training_error = np.sum(mistakes[:,-1])
print(f"total-training-error=",total_training_error)

total-training-error= 78.0


In [ ]:
# @title Evaluation on Testing Dataset
test_data = X_test.T
test_true_label = y_test
mistakes_test = 0
w = w.squeeze()
for i in range(test_data.shape[1]):
  predicted_test_label = np.argmax(w@test_data[:,i]+ b)
  if test_true_label[i] != predicted_test_label+1:
    mistakes_test+=1
print("Total mistakes-in-test-data",mistakes_test)

Total mistakes-in-test-data 145
